In [ ]:
import os
from experiment_setup import setups

In [ ]:
# download the dataset, the links can be fund in the README
dataset_path = "./datasets/coveo_ecommerce"
model_path = "./trained_models"

## Run the preprocess script, specific to the dataset you chose

- The preprocessing script in general, executes the following steps:
    - Loads the raw data, with correct types
    - Creates the sessions
    - Removes duplicated items. An item is considered as a duplicate if the preceding (based on time) event in the same session contains the exact same item.
    - Performes iterative support filtering
        - Removes sessions with only one event
        - Removes items with less than 5 events
        - Until the size of the dataset changes


In [ ]:
%run coveo_preproc.py --path $dataset_path

## Use a specific setup for your dataset

In [ ]:
params = setups["coveo"]["params_bprmax"]

In [ ]:
train_path = os.path.join(dataset_path,"coveo_processed_view_train_full.tsv")
test_path = os.path.join(dataset_path,"coveo_processed_view_test.tsv")

In [ ]:
def create_torch_gru4rec_script(model_name, train_path, test_path, model_path, loss, optim, final_act, layers, batch_size, dropout_p_embed, dropout_p_hidden, learning_rate, sample_alpha, bpreg, n_epochs, n_sample, m):
    s_train_full = f"CUDA_VISIBLE_DEVICES=0 python ./Torch-GRU4Rec/main.py --save_path {model_path}/{model_name} --train_path {train_path} --loss {'nll' if loss =='cross-entropy' else loss} --optimizer {optim} --n_epochs {n_epochs} --embedding_size {layers} --hidden_size {layers} --n_layers {1} --final_act {'softmaxlogit' if final_act=='softmax' else final_act} --batch_size {batch_size} --dropout_p_embed {dropout_p_embed} --dropout_p_hidden {dropout_p_hidden} --lr {learning_rate} --n_sample {n_sample} --sample_alpha {sample_alpha} --bpreg {bpreg}"
    s_test_full = f"CUDA_VISIBLE_DEVICES=0 python /db_vol/aczapp/sandbox/third_parties/Torch-GRU4Rec/main.py --train_path {train_path} --test_path {test_path} --model_path {model_path}/{model_name}/model_0000{n_epochs-1}.pt --test  --m {m}"
    return s_train_full, s_test_full

In [ ]:
loss = params["loss"]
optim = params["optim"]
const_emb = params["constrained_embedding"]
embed = params["embedding"]
final_act = params["final_act"]
layers = params["layers"]
batch_size = params["batch_size"]
dropout_p_embed = params["dropout_p_embed"]
dropout_p_hidden = params["dropout_p_hidden"]
learning_rate = params["learning_rate"]
momentum = params["momentum"]
sample_alpha = params["sample_alpha"]
bpreg = params["bpreg"]
logq = params["logq"]
hidden_act = params["hidden_act"]
n_epochs = 5
n_sample = 2048
m = '1 5 10 20'

## Train & test the out-of-the-box model

In [ ]:
train_script_oob, test_script_oob = create_torch_gru4rec_script(model_name='torch_gru4rec_oob_bprmax', train_path=train_path, test_path=test_path, model_path=model_path, loss=loss, optim=optim, final_act=final_act, layers=layers, batch_size=batch_size, dropout_p_embed=dropout_p_embed, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, sample_alpha=sample_alpha, bpreg=bpreg, n_epochs=n_epochs, n_sample=n_sample, m=m)

### Train the out-of-the-box model

In [ ]:
os.system(train_script_oob)

### Test the out-of-the-box model

In [ ]:
os.system(test_script_oob)